Big Data Machine Learning, Distributed Machine Learning, Parallel Machine Learning with C++, CUDA, Scala, Spark
=============

Ron Wu
-------------

10/21/16

Reference: free courses by the creators of
<br>NVIDIA https://developer.nvidia.com/udacity-cs344-intro-parallel-programming<br>
Scala http://www.scala-lang.org/blog/2016/05/23/scala-moocs-specialization-launched.html<br>
Spark, Databricks https://databricks.com/blog/2016/06/01/databricks-to-launch-first-of-five-free-big-data-courses-on-apache-spark.html <br> 
### Contents


1. <a href =#scala>Functional and Object-Oriented Scala</a>
     - <a href=#scalab>Functional Aspect of Scala</a>
     - <a href = #relu>Data-Parallelism</a> 
     - <a href = #relu>Scala with Spark</a> 
<br><br>
2. <a href=#Spark>Big Data, Distributed Analysis with Spark</a>
     - <a href =#entropy>Spark with Time Series: spark-ts</a> 
     - <a href =#entropy>Spark with TensorFlow: TensorFrames</a> 
     - <a href =#entropy>Other Spark Ecosystem</a> 
<br><br>
3. <a href =#convnet>Parallel with CUDA, C++</a>
     - <a href=#worked>Parallel Computing</a> 
     - <a href=#worked>GPU Programming</a> 
<br><br>
 

<a name = 'scala'></a>
# Functional and Object-Oriented Scala

<a name = 'scalab'></a>
## Functional Aspect of Scala - Thread Safe

For easier transition to parallel programming, throughout this notebook I will try to use functional style over serial style whenever possible. That is because in parallel programming variables are immutable. Once it is set, it cannot be changed, which avoids a lot deadlocks. 

If a block of code is written in loop fashion, the compiler will turn it into functions, the iteration and any change of variables inside the loop are passing through the function calls. Thus whether you write it in serial or recursion, it will end up in recursive call anyway.  


In [1]:
%%file main.scala

object main extends App {

  def fib(n: Int) = { 
        //recursive function has to have explicit return type
        // : is optional unless {} is not used or not on the same line
      def fibIter(i: Int, a: Int, b: Int): Int =
          if (i == n) a else fibIter(i+1, b, a+b) // that is tail recursive, hence will not build up the stack
      fibIter(0, 0, 1)
  }

  println(fib(10))
}

Overwriting main.scala


In [2]:
! scalac main.scala
! scala main

55


<a name='Spark'></a>
# Big Data with Spark

- Apache Spark Components 
    - Spark SQL
    - Spark Streaming
    - MLlib
    - GraphX